# Audience Decode – ML Classification

Predict whether a user will give a **high rating** (≥ 4) to a movie interaction.

**Data Split:** Uses pre-split databases (random split):
- `viewer_interactions_train.db` — 70% training data
- `viewer_interactions_val.db` — 15% validation data
- `viewer_interactions_test.db` — 15% test data

**Models:**
- Logistic Regression
- Random Forest
- Gradient Boosting


In [3]:
import sqlite3  # connect to SQLite database
from pathlib import Path  # handle filesystem paths

import numpy as np  # numerical computations
import pandas as pd  # data manipulation

from sklearn.compose import ColumnTransformer  # apply different transforms to column subsets
from sklearn.pipeline import Pipeline  # chain preprocessing and model
from sklearn.preprocessing import StandardScaler, OneHotEncoder  # feature scaling and encoding
from sklearn.impute import SimpleImputer  # handle missing values

from sklearn.linear_model import LogisticRegression  # logistic regression classifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier  # tree-based classifiers

from sklearn.metrics import (  # evaluation metrics
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
    confusion_matrix,
)

import matplotlib.pyplot as plt  # plotting

# Paths to pre-split databases (70% train, 15% val, 15% test)
TRAIN_DB_PATH = Path("./viewer_interactions_train.db")
VAL_DB_PATH = Path("./viewer_interactions_val.db")
TEST_DB_PATH = Path("./viewer_interactions_test.db")

assert TRAIN_DB_PATH.exists(), f"Training DB not found at {TRAIN_DB_PATH}"
assert VAL_DB_PATH.exists(), f"Validation DB not found at {VAL_DB_PATH}"
assert TEST_DB_PATH.exists(), f"Test DB not found at {TEST_DB_PATH}"


In [4]:
def load_data_from_db(db_path):
    """Load viewer_ratings, user_statistics, movie_statistics from a database."""
    conn = sqlite3.connect(db_path)
    viewer_ratings = pd.read_sql("SELECT * FROM viewer_ratings;", conn)
    user_statistics = pd.read_sql("SELECT * FROM user_statistics;", conn)
    movie_statistics = pd.read_sql("SELECT * FROM movie_statistics;", conn)
    conn.close()
    return viewer_ratings, user_statistics, movie_statistics

# Load training data
train_ratings, train_user_stats, train_movie_stats = load_data_from_db(TRAIN_DB_PATH)
print("Training data:")
print(f"  viewer_ratings: {train_ratings.shape}")
print(f"  user_statistics: {train_user_stats.shape}")
print(f"  movie_statistics: {train_movie_stats.shape}")

# Load validation data
val_ratings, val_user_stats, val_movie_stats = load_data_from_db(VAL_DB_PATH)
print("\nValidation data:")
print(f"  viewer_ratings: {val_ratings.shape}")
print(f"  user_statistics: {val_user_stats.shape}")
print(f"  movie_statistics: {val_movie_stats.shape}")

# Load test data
test_ratings, test_user_stats, test_movie_stats = load_data_from_db(TEST_DB_PATH)
print("\nTest data:")
print(f"  viewer_ratings: {test_ratings.shape}")
print(f"  user_statistics: {test_user_stats.shape}")
print(f"  movie_statistics: {test_movie_stats.shape}")


Training data:
  viewer_ratings: (2817500, 5)
  user_statistics: (405158, 10)
  movie_statistics: (12980, 11)

Validation data:
  viewer_ratings: (603750, 5)
  user_statistics: (238836, 10)
  movie_statistics: (4170, 11)

Test data:
  viewer_ratings: (603750, 5)
  user_statistics: (238412, 10)
  movie_statistics: (4180, 11)


In [5]:
def preprocess_ratings(df):
    """Clean and preprocess ratings data, add target label."""
    df = df.copy()
    df["rating"] = pd.to_numeric(df["rating"], errors="coerce")  # ensure rating is numeric
    df["date"] = pd.to_datetime(df["date"], errors="coerce")  # ensure date is datetime
    
    mask_valid = df["rating"].between(1, 5)  # allow only ratings 1..5
    df = df.loc[mask_valid].copy()  # keep only valid ratings
    
    if "anomalous_date" in df.columns:  # if anomalous_date exists
        df = df[df["anomalous_date"] != 1].copy()  # drop rows with anomalous_date == 1
    
    df = df.dropna(subset=["date"]).copy()  # drop rows with missing date
    df["label_high"] = (df["rating"] >= 4).astype(int)  # 1 if rating >= 4, else 0
    return df

# Preprocess all three datasets
train_df = preprocess_ratings(train_ratings)
val_df = preprocess_ratings(val_ratings)
test_df = preprocess_ratings(test_ratings)

print("Training label distribution:")
print(train_df["label_high"].value_counts(normalize=True))
print(f"\nValidation label distribution:")
print(val_df["label_high"].value_counts(normalize=True))
print(f"\nTest label distribution:")
print(test_df["label_high"].value_counts(normalize=True))


Training label distribution:
label_high
1    0.572452
0    0.427548
Name: proportion, dtype: float64

Validation label distribution:
label_high
1    0.573459
0    0.426541
Name: proportion, dtype: float64

Test label distribution:
label_high
1    0.573636
0    0.426364
Name: proportion, dtype: float64


In [6]:
def prepare_stats(user_statistics, movie_statistics):
    """Rename columns in user and movie statistics for merging."""
    user_stats = user_statistics.copy()
    movie_stats = movie_statistics.copy()
    
    user_rename = {
        "total_ratings": "user_total_ratings",
        "unique_movies": "user_unique_movies",
        "avg_rating": "user_avg_rating",
        "std_rating": "user_std_rating",
        "activity_days": "user_activity_days",
    }
    user_stats = user_stats.rename(columns=user_rename)
    
    movie_rename = {
        "total_ratings": "movie_total_ratings",
        "unique_users": "movie_unique_users",
        "avg_rating": "movie_avg_rating",
        "year_of_release": "movie_year_of_release",
    }
    movie_stats = movie_stats.rename(columns=movie_rename)
    
    return user_stats, movie_stats

# Prepare stats for each split
train_user_stats_renamed, train_movie_stats_renamed = prepare_stats(train_user_stats, train_movie_stats)
val_user_stats_renamed, val_movie_stats_renamed = prepare_stats(val_user_stats, val_movie_stats)
test_user_stats_renamed, test_movie_stats_renamed = prepare_stats(test_user_stats, test_movie_stats)

print("Training user stats shape:", train_user_stats_renamed.shape)
print("Training movie stats shape:", train_movie_stats_renamed.shape)


Training user stats shape: (405158, 10)
Training movie stats shape: (12980, 11)


In [7]:
def merge_interactions(df, user_stats, movie_stats):
    """Merge ratings with user and movie statistics."""
    interactions = df[["customer_id", "movie_id", "rating", "date", "label_high"]].copy()
    
    interactions = interactions.merge(
        user_stats,
        how="inner",
        on="customer_id",
    )
    
    interactions = interactions.merge(
        movie_stats,
        how="inner",
        on="movie_id",
    )
    
    return interactions

# Merge all three splits
train_interactions = merge_interactions(train_df, train_user_stats_renamed, train_movie_stats_renamed)
val_interactions = merge_interactions(val_df, val_user_stats_renamed, val_movie_stats_renamed)
test_interactions = merge_interactions(test_df, test_user_stats_renamed, test_movie_stats_renamed)

print("Train merged shape:", train_interactions.shape)
print("Val merged shape:", val_interactions.shape)
print("Test merged shape:", test_interactions.shape)


Train merged shape: (2534660, 24)
Val merged shape: (542941, 24)
Test merged shape: (543038, 24)


In [8]:
def add_features(interactions):
    """Add engineered features to interactions dataframe."""
    df = interactions.copy()
    
    df["rating_year"] = df["date"].dt.year  # extract rating year
    df["rating_month"] = df["date"].dt.month  # extract rating month
    df["rating_dayofweek"] = df["date"].dt.weekday  # extract weekday
    
    df["movie_year_of_release"] = pd.to_numeric(
        df["movie_year_of_release"], errors="coerce"
    )
    df["movie_age_at_rating"] = df["rating_year"] - df["movie_year_of_release"]
    
    df["user_activity_days"] = pd.to_numeric(
        df["user_activity_days"], errors="coerce"
    )
    activity_days_safe = df["user_activity_days"].replace(0, np.nan)
    df["user_ratings_per_month"] = df["user_total_ratings"] / (activity_days_safe / 30.0)
    
    return df

# Add features to all splits
train_interactions = add_features(train_interactions)
val_interactions = add_features(val_interactions)
test_interactions = add_features(test_interactions)

print("Sample features from training data:")
print(train_interactions[["user_total_ratings", "user_activity_days", "user_ratings_per_month"]].head())


Sample features from training data:
   user_total_ratings  user_activity_days  user_ratings_per_month
0                   7                 127                1.653543
1                  18                1113                0.485175
2                  43                 619                2.084006
3                   4                  15                8.000000
4                  13                  73                5.342466


In [9]:
numeric_features = [  # numeric columns
    "user_total_ratings",
    "user_unique_movies",
    "user_avg_rating",
    "user_std_rating",
    "user_activity_days",
    "user_ratings_per_month",
    "movie_total_ratings",
    "movie_unique_users",
    "movie_avg_rating",
    "movie_year_of_release",
    "movie_age_at_rating",
    "rating_year",
]

categorical_features = [  # categorical columns
    "rating_month",
    "rating_dayofweek",
]

def prepare_X_y(interactions, numeric_features, categorical_features):
    """Prepare feature matrix X and target y from interactions."""
    model_df = interactions[numeric_features + categorical_features + ["label_high"]].copy()
    model_df = model_df.dropna(subset=["label_high"])
    
    X = model_df[numeric_features + categorical_features].copy()
    y = model_df["label_high"].astype(int).copy()
    return X, y

# Prepare X, y for all splits
X_train, y_train = prepare_X_y(train_interactions, numeric_features, categorical_features)
X_val, y_val = prepare_X_y(val_interactions, numeric_features, categorical_features)
X_test, y_test = prepare_X_y(test_interactions, numeric_features, categorical_features)

print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape, "y_val shape:", y_val.shape)
print("X_test shape:", X_test.shape, "y_test shape:", y_test.shape)


X_train shape: (2534660, 14) y_train shape: (2534660,)
X_val shape: (542941, 14) y_val shape: (542941,)
X_test shape: (543038, 14) y_test shape: (543038,)


In [10]:
# Data is already split from separate databases (70% train, 15% val, 15% test)
# No need to use train_test_split - we use the pre-split databases

print("=== Dataset Split Summary ===")
print(f"Train size: {X_train.shape[0]:,} ({X_train.shape[0] / (X_train.shape[0] + X_val.shape[0] + X_test.shape[0]) * 100:.1f}%)")
print(f"Val size: {X_val.shape[0]:,} ({X_val.shape[0] / (X_train.shape[0] + X_val.shape[0] + X_test.shape[0]) * 100:.1f}%)")
print(f"Test size: {X_test.shape[0]:,} ({X_test.shape[0] / (X_train.shape[0] + X_val.shape[0] + X_test.shape[0]) * 100:.1f}%)")
print(f"\nTotal: {X_train.shape[0] + X_val.shape[0] + X_test.shape[0]:,}")


=== Dataset Split Summary ===
Train size: 2,534,660 (70.0%)
Val size: 542,941 (15.0%)
Test size: 543,038 (15.0%)

Total: 3,620,639


In [11]:
numeric_transformer = Pipeline(  # pipeline for numeric features
    steps=[
        ("imputer", SimpleImputer(strategy="median")),  # fill missing values with median
        ("scaler", StandardScaler()),  # standardize numeric features
    ]
)

categorical_transformer = Pipeline(  # pipeline for categorical features
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),  # fill missing with most frequent
        ("onehot", OneHotEncoder(handle_unknown="ignore")),  # one-hot encode categories
    ]
)

preprocessor = ColumnTransformer(  # apply transformers to columns
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)


In [12]:
log_reg = LogisticRegression(  # logistic regression model
    max_iter=1000,
    class_weight="balanced",
)

rf_clf = RandomForestClassifier(  # random forest classifier
    n_estimators=100,
    random_state=42,
    n_jobs=-1,
)

gb_clf = GradientBoostingClassifier(  # gradient boosting classifier
    random_state=42,
)

models = {  # dict of model pipelines
    "Logistic Regression": Pipeline([
        ("preprocess", preprocessor),
        ("model", log_reg),
    ]),
    "Random Forest": Pipeline([
        ("preprocess", preprocessor),
        ("model", rf_clf),
    ]),
    "Gradient Boosting": Pipeline([
        ("preprocess", preprocessor),
        ("model", gb_clf),
    ]),
}


In [13]:
val_results = []  # list to store validation metrics

for name, pipe in models.items():  # loop over models
    print("\n====================")  # separator
    print("Training:", name)  # show model name
    pipe.fit(X_train, y_train)  # fit model on train set

    y_val_pred = pipe.predict(X_val)  # predict labels on val set
    y_val_proba = pipe.predict_proba(X_val)[:, 1]  # predict proba for class 1

    acc = accuracy_score(y_val, y_val_pred)  # accuracy
    prec = precision_score(y_val, y_val_pred, zero_division=0)  # precision
    rec = recall_score(y_val, y_val_pred, zero_division=0)  # recall
    f1 = f1_score(y_val, y_val_pred, zero_division=0)  # F1-score
    roc_auc = roc_auc_score(y_val, y_val_proba)  # ROC-AUC

    print(f"Val Accuracy: {acc:.4f}")  # show accuracy
    print(f"Val Precision: {prec:.4f}")  # show precision
    print(f"Val Recall: {rec:.4f}")  # show recall
    print(f"Val F1: {f1:.4f}")  # show F1
    print(f"Val ROC-AUC: {roc_auc:.4f}")  # show ROC-AUC

    print("Classification report (val):")  # header
    print(classification_report(y_val, y_val_pred, zero_division=0))  # detailed report

    val_results.append({  # store metrics
        "model": name,
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "roc_auc": roc_auc,
    })

val_results_df = pd.DataFrame(val_results)  # metrics table
print("\nValidation metrics:")  # header
print(val_results_df)  # show metrics



Training: Logistic Regression
Val Accuracy: 0.8029
Val Precision: 0.8471
Val Recall: 0.8008
Val F1: 0.8233
Val ROC-AUC: 0.8849
Classification report (val):
              precision    recall  f1-score   support

           0       0.75      0.81      0.78    231586
           1       0.85      0.80      0.82    311355

    accuracy                           0.80    542941
   macro avg       0.80      0.80      0.80    542941
weighted avg       0.81      0.80      0.80    542941


Training: Random Forest
Val Accuracy: 0.8169
Val Precision: 0.8314
Val Recall: 0.8538
Val F1: 0.8424
Val ROC-AUC: 0.9130
Classification report (val):
              precision    recall  f1-score   support

           0       0.80      0.77      0.78    231586
           1       0.83      0.85      0.84    311355

    accuracy                           0.82    542941
   macro avg       0.81      0.81      0.81    542941
weighted avg       0.82      0.82      0.82    542941


Training: Gradient Boosting
Val Accur

In [14]:
best_idx = val_results_df["f1"].idxmax()  # index of best model by F1
best_name = val_results_df.loc[best_idx, "model"]  # name of best model
print("Best model:", best_name)  # show best model

best_pipe = models[best_name]  # corresponding pipeline

X_train_full = pd.concat([X_train, X_val], axis=0)  # merge train and val features
y_train_full = pd.concat([y_train, y_val], axis=0)  # merge train and val labels

best_pipe.fit(X_train_full, y_train_full)  # refit on full train data

y_test_pred = best_pipe.predict(X_test)  # predict labels on test set
y_test_proba = best_pipe.predict_proba(X_test)[:, 1]  # predict proba on test set

test_acc = accuracy_score(y_test, y_test_pred)  # test accuracy
test_prec = precision_score(y_test, y_test_pred, zero_division=0)  # test precision
test_rec = recall_score(y_test, y_test_pred, zero_division=0)  # test recall
test_f1 = f1_score(y_test, y_test_pred, zero_division=0)  # test F1
test_roc_auc = roc_auc_score(y_test, y_test_proba)  # test ROC-AUC

print("\n=== Test performance ===")  # header
print(f"Test Accuracy: {test_acc:.4f}")  # show accuracy
print(f"Test Precision: {test_prec:.4f}")  # show precision
print(f"Test Recall: {test_rec:.4f}")  # show recall
print(f"Test F1: {test_f1:.4f}")  # show F1
print(f"Test ROC-AUC: {test_roc_auc:.4f}")  # show ROC-AUC

print("\nClassification report (test):")  # header
print(classification_report(y_test, y_test_pred, zero_division=0))  # detailed report

print("Confusion matrix (test):")  # header
print(confusion_matrix(y_test, y_test_pred))  # confusion matrix


Best model: Gradient Boosting

=== Test performance ===
Test Accuracy: 0.8181
Test Precision: 0.8299
Test Recall: 0.8589
Test F1: 0.8441
Test ROC-AUC: 0.9147

Classification report (test):
              precision    recall  f1-score   support

           0       0.80      0.76      0.78    231532
           1       0.83      0.86      0.84    311506

    accuracy                           0.82    543038
   macro avg       0.82      0.81      0.81    543038
weighted avg       0.82      0.82      0.82    543038

Confusion matrix (test):
[[176676  54856]
 [ 43949 267557]]
